<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch08/section_70.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 単語埋め込みの読み込み
事前学習済みの単語ベクトルを読み込んで、行列Eを作成する
条件
- E[0]には\<PAD>用にゼロベクトル
- E[1:]以降に事前学習済みベクトルを順に格納
- 同時にtoken ↔︎ token_idの対応（双方向対応）を保持

### 具体的な説明
1. 行列Eとはどうなっているか
   - Eは「単語ID -> 単語ベクトル」を引ける表
   - サイズは（V * d）
     - V = 語彙数（トークン数）
     - d = 埋め込み次元（1単語あたりのベクトルの長さ）
2. PADとは
   - 自然言語処理では、文の長さがバラバラなので、見にバッチ処理を追加するために長さを揃える必要がある。
   - そのときに埋めるtokenが\<PAD>（padding token）である。  
   - なぜPADをゼロベクトルにするか
     1. 計算に影響を与えにくい（無害にしたい）
        - \<PAD>は本当の単語ではなく穴埋めなのでモデルの判断に影響させたくない
        - ゼロベクトルなら、後段で足し算や平均を取ったときに邪魔しにくい
     2. マスク・無視処理がしやすい
        - 多くのライブラリや実装では「padding_idx=0（ID=0はPAD）」みたいに0番を特別扱いにする記法がよくある
        - 先頭行をPADに予約すると実装がシンプルになる
     3. PADを学習で更新しないようにできる
        - PyTorchのnn.Embedding(..., padding_idx=0)は、PAD行をほぼ固定できる。
3. 双方向紐付けとは
   - token2id: 単語（トークン）-> 行列Eの行番号（ID）
   - id2token: 行番号（ID）-> 単語（トークン）

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install gensim # colabにはデフォルトで入っていないのでinstallする

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 77.7 MB/s eta 0:00:00


### KeyedVectorsとは何をしている？
- 学習済みの「単語（キー）→ベクトル」の対応を保持するクラス
- 単語を指定すると対応する埋め込みベクトルを取り出せる（参照できる）
- 学習機能は持たず、読み込み・参照に特化している


In [4]:
from gensim.models import KeyedVectors
import numpy as np

In [ ]:
wv_src = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz' # gzipはデータの中の繰り返しパターンを短い表現に置き換える
wv = KeyedVectors.load_word2vec_format(wv_src, binary=True) # binary=Trueでバイナリ形式（0, 1のバイト列）で対応できるようにする

### len() で語彙数が分かる理由
`len(wv)` を呼び出すと、内部では `wv.__len__()` が実行される。

gensim の `KeyedVectors` クラスでは、`__len__()` メソッドが
内部の `key_to_index`（単語 → インデックスの辞書）の長さを
返すように実装されている。

```python
def __len__(self):
    return len(self.key_to_index)
```

### byteの単位

```
1 KB = 1024 bytes
1 MB = 1024 KB
1 GB = 1024 MB
```




In [ ]:
V = len(wv) + 1 # +1は<PAD>分
print('語彙数:', V)

d = wv.vector_size
print('埋め込み次元数', d)

print(wv.vectors.shape) # GoogleNews-vectors-negative300の埋め込み行列
print(wv.vectors.nbytes / 1024**3) # 使用メモリ数

In [ ]:
import torch

torch.zeros()はこの場合、(V * d)の行列を作り、全部ゼロで埋めている
dtype=torch.float32は各要素のデータ型でfloat32は32ビット浮動小数点数

In [ ]:
E = torch.zeros((V, d), dtype=torch.float32) # PyTorchのニューラルネットが期待している型がfloat32らしい。今回は学習範囲外
token2id = {"<PAD>": 0}
id2token = ["<PAD>"]

### torch.tensorとtorch.from_numpyの違い
- `torch.tensor(...)`
  - tensor型に変換する
  - 基本的にメモリを新しく確保（コピーする）
- `torch.from_numpy(...)`
  - Numpy配列 -> tensor型に変換する
  - メモリを複製しない（Numpy配列とメモリ共有）

In [ ]:
for i, token in enumerate(wv.index_to_key, start=1): # すでに0のindexはPADに割り当てているので1から開始する
  token2id[token] = i
  id2token.append(token)
  E[i] = torch.tensor(wv[token]) # from_numpyはnumpy配列をPyTorchテンソル型に変換する関数（torch.Tensor）

In [ ]:
print("=== Embedding matrix info ===")
print("E shape:", E.shape)           # (V, d) になっているか
print("Embedding dim (d):", E.shape[1])
print("Vocab size (V):", E.shape[0])
print(f"idx2token[1]:{id2token[1]}")